<a href="https://colab.research.google.com/github/starryesh22/Google_Colab/blob/main/0607_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm
import numpy as np

from dataset import ECGDataset
from resnet import resnet34
from utils import cal_f1s, cal_aucs, split_data

'''

# 신경망 모델을 훈련하고 평가 작업 준비

argparse: 명령줄 인수를 처리하기 위한 모듈
os: 운영 체제와 상호 작용하기 위한 모듈
torch: PyTorch 라이브러리
torch.nn: 신경망 모델을 구성하기 위한 클래스와 함수를 제공하는 모듈
torch.utils.data.DataLoader: 데이터셋을 미니배치로 나누고 데이터를 로드하기 위한 클래스
tqdm: 반복문의 진행 상태를 시각화하는 라이브러리
numpy: 수치 연산을 위한 패키지
또한 dataset 모듈에서 ECGDataset 클래스를 가져오고, 
resnet 모듈에서 resnet34 모델을 가져오며, 
utils 모듈에서 몇 가지 유틸리티 함수를 가져옴

'''


In [ ]:
# parse_args 함수

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='data/CPSC', help='Directory for data dir')
    parser.add_argument('--leads', type=str, default='all', help='ECG leads to use')
    parser.add_argument('--seed', type=int, default=42, help='Seed to split data')
    parser.add_argument('--num-classes', type=int, default=int, help='Num of diagnostic classes')
    parser.add_argument('--lr', '--learning-rate', type=float, default=0.0001, help='Learning rate')
    parser.add_argument('--batch-size', type=int, default=32, help='Batch size')
    parser.add_argument('--num-workers', type=int, default=4, help='Num of workers to load data')
    parser.add_argument('--phase', type=str, default='train', help='Phase: train or test')
    parser.add_argument('--epochs', type=int, default=40, help='Training epochs')
    parser.add_argument('--resume', default=False, action='store_true', help='Resume')
    parser.add_argument('--use-gpu', default=False, action='store_true', help='Use GPU')
    parser.add_argument('--model-path', type=str, default='', help='Path to saved model')
    return parser.parse_args()


''' 

코드는 parse_args라는 함수로 정의되어 있음
이 함수는 명령줄 인수를 파싱하여 해당 인수들을 반환
parse_args 함수의 동작을 설명함.

argparse.ArgumentParser를 사용하여 parser 객체를 생성
--data-dir 인수를 추가하여 데이터 디렉토리를 지정, 기본값은 'data/CPSC'
--leads 인수를 추가하여 사용할 ECG 리드(심전도 리드)를 지정, 기본값은 'all'
--seed 인수를 추가하여 데이터 분할을 위한 시드(seed)를 지정, 기본값은 42
--num-classes 인수를 추가하여 진단 클래스의 수를 지정,  int로 지정
--lr 또는 --learning-rate 인수를 추가하여 학습률(learning rate)을 지정, 기본값은 0.0001
--batch-size 인수를 추가하여 미니배치 크기를 지정, 기본값은 32
--num-workers 인수를 추가하여 데이터 로딩을 위한 워커(worker)의 수를 지정, 기본값은 4
--phase 인수를 추가하여 훈련 또는 테스트 단계를 지정, 기본값은 'train'
--epochs 인수를 추가하여 훈련할 epoch 수를 지정, 기본값은 40
--resume 인수를 추가하여 이전에 저장한 모델을 재개하여 훈련을 지속함. 기본값은 False
--use-gpu 인수를 추가하여 GPU를 사용할지 여부를 지정함. 기본값은 False
--model-path 인수를 추가하여 저장된 모델의 경로를 지정. 기본값은 '빈 문자열'
parser.parse_args()를 호출하여 인수를 파싱,  파싱한 인수를 반환
- 명령줄에서 지정한 인수들을 간편하게 접근 가능

'''

In [ ]:
# train 함수 (훈련 함수)

def train(dataloader, net, args, criterion, epoch, scheduler, optimizer, device):
    print('Training epoch %d:' % epoch)
    net.train()
    running_loss = 0
    output_list, labels_list = [], []
    for _, (data, labels) in enumerate(tqdm(dataloader)):
        data, labels = data.to(device), labels.to(device)
        output = net(data)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        output_list.append(output.data.cpu().numpy())
        labels_list.append(labels.data.cpu().numpy())
    # scheduler.step()
    print('Loss: %.4f' % running_loss)
    
''' 

주어진 데이터로더를 사용하여 신경망 모델을 훈련하는 작업을 수행.
- train 함수의 동작을 설명하는 부분 :

현재 실행 중인 epoch 값을 출력
모델을 훈련 모드로 설정
running_loss 변수를 초기화
output_list와 labels_list라는 빈 리스트를 생성
(훈련 중에 출력과 실제 레이블을 저장하기 위한 리스트) 
데이터로더를 순회하면서 데이터와 레이블을 가져옴.
데이터와 레이블을 디바이스(GPU 또는 CPU)로 전송
모델에 데이터를 입력하여 출력을 얻음
손실 함수를 사용하여 출력과 레이블 간의 손실을 계산
옵티마이저의 그래디언트를 초기화함.
손실에 대한 그래디언트를 계산
옵티마이저를 사용하여 모델의 가중치를 업데이트함.
손실 값을 running_loss에 더함.
출력과 레이블을 output_list와 labels_list에 추가함.
running_loss 값을 출력.
train 함수는 주어진 데이터로더를 사용하여 
한 epoch 동안 신경망 모델을 훈련하고, 훈련 손실을 출력함.

'''

In [ ]:
# evalutate 함수  (신경망 모델 평가)

def evaluate(dataloader, net, args, criterion, device):
    print('Validating...')
    net.eval()
    running_loss = 0
    output_list, labels_list = [], []
    for _, (data, labels) in enumerate(tqdm(dataloader)):
        data, labels = data.to(device), labels.to(device)
        output = net(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        output = torch.sigmoid(output)
        output_list.append(output.data.cpu().numpy())
        labels_list.append(labels.data.cpu().numpy())
    print('Loss: %.4f' % running_loss)
    y_trues = np.vstack(labels_list)
    y_scores = np.vstack(output_list)
    f1s = cal_f1s(y_trues, y_scores)
    avg_f1 = np.mean(f1s)
    print('F1s:', f1s)
    print('Avg F1: %.4f' % avg_f1)
    if args.phase == 'train' and avg_f1 > args.best_metric:
        args.best_metric = avg_f1
        torch.save(net.state_dict(), args.model_path)
    else:
        aucs = cal_aucs(y_trues, y_scores)
        avg_auc = np.mean(aucs)
        print('AUCs:', aucs)
        print('Avg AUC: %.4f' % avg_auc)

''' 

평가 함수인 evaluate 함수로 주어진 데이터로더를 사용하여 신경망 모델을 평가하는 작업을 수행
evaluate 함수의 동작을 설명

- 'Validating...' 메시지를 출력.
- 모델을 평가 모드로 설정
- running_loss 변수를 초기화
- output_list와 labels_list라는 빈 리스트를 생성
- 이들은 평가 중에 출력과 실제 레이블을 저장하기 위한 리스트임
- 데이터로더를 순회하면서 데이터와 레이블을 가져옴
- 데이터와 레이블을 디바이스(GPU 또는 CPU)로 전송합니다.
- 모델에 데이터를 입력하여 출력을 얻습니다.
- 손실 함수를 사용하여 출력과 레이블 간의 손실을 계산합니다.
- 손실 값을 running_loss에 더합니다.
- 출력에 시그모이드 함수를 적용하여 확률 값을 얻음.
- 출력과 레이블을 output_list와 labels_list에 추가
- running_loss 값을 출력함.
- labels_list와 output_list를 NumPy 배열로 변환하여 실제 레이블과 예측 확률을 얻음
- cal_f1s 함수를 사용하여 F1 점수를 계산.
- 평균 F1 점수를 계산.
- F1 점수와 평균 F1 점수를 출력.
- 훈련 단계일 때, 평균 F1 점수가 최고 점수(args.best_metric)보다 높으면 모델을 저장.
- 테스트 단계일 때, AUC 값을 계산하고 출력함.
- evaluate 함수는 주어진 데이터로더를 사용하여 신경망 모델을 평가함
- 평가 손실 및 F1 점수 또는 AUC 값을 출력함.
- 훈련 단계일 때 최고 점수를 가진 모델을 저장

'''

In [ ]:
# main 함수 

if __name__ == "__main__":
    args = parse_args()
    args.best_metric = 0
    data_dir = os.path.normpath(args.data_dir)
    database = os.path.basename(data_dir)

    if not args.model_path:
        args.model_path = f'models/resnet34_{database}_{args.leads}_{args.seed}.pth'

    if args.use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = 'cpu'
    
    if args.leads == 'all':
        leads = 'all'
        nleads = 12
    else:
        leads = args.leads.split(',')
        nleads = len(leads)
    
''' 

메인 함수인 __main__ 함수입니다. 이 함수는 다음 작업을 수행합니다:

- parse_args 함수를 사용하여 명령줄 인수를 파싱하고, 반환된 인수를 args 변수에 할당합니다.
- args.best_metric 변수를 0으로 초기화합니다.
- args.data_dir을 정규화하여 data_dir 변수에 할당합니다.
- data_dir의 베이스 이름을 추출하여 database 변수에 할당합니다.
- 만약 args.model_path가 주어지지 않았다면, 기본 경로를 생성하여 args.model_path에 할당합니다. 
- 생성된 경로는 models/resnet34_{database}_{args.leads}_{args.seed}.pth 형식입니다.
- 만약 args.use_gpu가 True이고 CUDA가 사용 가능한 경우, 디바이스를 CUDA 디바이스로 설정합니다. 
  그렇지 않으면 CPU를 사용합니다.
- 만약 args.leads가 'all'인 경우, leads를 'all'로 설정하고 nleads를 12로 설정합니다. 
- 그렇지 않으면, leads를 쉼표로 분리하여 리스트로 만들고, nleads를 leads의 길이로 설정합니다.
- __main__ 함수는 명령줄 인수를 파싱하고, 모델 경로와 디바이스를 설정합니다. 
  또한 leads와 nleads 변수를 설정하여 ECG 리드 정보를 지정합니다.

'''

In [ ]:
# main 함수 나머지 부분.

    label_csv = os.path.join(data_dir, 'labels.csv')
    
    train_folds, val_folds, test_folds = split_data(seed=args.seed)
    train_dataset = ECGDataset('train', data_dir, label_csv, train_folds, leads)
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, 
                              num_workers=args.num_workers, pin_memory=True)
    val_dataset = ECGDataset('val', data_dir, label_csv, val_folds, leads)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, 
                            num_workers=args.num_workers, pin_memory=True)
    test_dataset = ECGDataset('test', data_dir, label_csv, test_folds, leads)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, 
                             num_workers=args.num_workers, pin_memory=True)
    net = resnet34(input_channels=nleads).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=args.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.1)
    
    criterion = nn.BCEWithLogitsLoss()
    
    if args.phase == 'train':
        if args.resume:
            net.load_state_dict(torch.load(args.model_path, map_location=device))
        for epoch in range(args.epochs):
            train(train_loader, net, args, criterion, epoch, scheduler, optimizer, device)
            evaluate(val_loader, net, args, criterion, device)
    else:
        net.load_state_dict(torch.load(args.model_path, map_location=device))
        evaluate(test_loader, net, args, criterion, device)


''' 

메인 함수인 __main__ 함수의 나머지 부분입니다. 

- label_csv 변수에 데이터 디렉토리와 'labels.csv' 파일의 경로를 할당
- split_data 함수를 사용하여 훈련, 검증, 테스트 데이터의 폴드를 분할하여 
   train_folds, val_folds, test_folds에 할당
- ECGDataset 클래스를 사용하여 훈련, 검증, 테스트 데이터셋을 생성
- DataLoader를 사용하여 훈련, 검증, 테스트 데이터로더를 생성
- resnet34 모델을 생성하고, 입력 채널 수를 nleads로 설정하고, 디바이스에 할당
- torch.optim.Adam 옵티마이저를 생성하고, 네트워크의 파라미터를 최적화
torch.optim.lr_scheduler.StepLR을 사용하여 학습률 스케줄러를 생성
nn.BCEWithLogitsLoss 손실 함수를 생성

만약 args.phase가 'train'인 경우:
args.resume이 True인 경우, 저장된 모델을 불러와서 네트워크에 로드
args.epochs 횟수만큼 반복문을 실행하면서 훈련과 검증을 수행
각 에포크마다 train 함수로 훈련을 진행하고, evaluate 함수로 검증을 수행
그렇지 않으면, 저장된 모델을 불러와서 네트워크에 로드하고, evaluate 함수로 테스트를 수행
__main__ 함수는 데이터 경로, 모델 경로, 데이터로더, 모델, 옵티마이저, 스케줄러, 손실 함수 등을 설정하고,
훈련 또는 테스트를 수행
훈련 단계에서는 훈련과 검증이 번갈아가면서 수행되며,  테스트 단계에서는 저장된 모델을 
로드하고 테스트를 수행합니다.

'''